In [4]:
# Data
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import impl.layer as l

# Dataset preparation and pre-processing
mnist = input_data.read_data_sets('data/MNIST_data/', one_hot=False)

X_train, y_train = mnist.train.images, mnist.train.labels
X_val, y_val = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels
# y_test.shape, y_val.shape, y_train.shape
# X_train.shape, X_train.dtype, X_val.shape, X_val.dtype, X_test.shape, X_test.dtype

Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
# Pre-processing: normalizing
def normalize(X):
    # max scale for images 255= 2**8= 8 bit grayscale for each channel
    return (X - X.mean(axis=0)) #/ X.std(axis=0)

X_train, X_val, X_test = normalize(X=X_train), normalize(X=X_val), normalize(X=X_test)

In [6]:
# Model
import impl.layer as l # or from impl.layer import *
from impl.loss import * # import all functions from impl.loss file # import impl.loss as loss_func
from sklearn.utils import shuffle as skshuffle

class FFNN:

    def __init__(self, D, C, H, L):
        self.L = L # number of layers or depth
        self.losses = {'train':[], 'valid':[], 'valid_acc':[]}
        
        # The adaptive/learnable/updatable random feedforward
        self.model = []
        self.W_fixed = []
        self.grads = []
        self.ys_prev = []
        low, high = -1, 1
        
        # Input layer: weights/ biases
        m = dict(W=np.random.uniform(size=(D, H), low=low, high=high) / np.sqrt(D / 2.), b=np.zeros((1, H)))
        self.model.append(m)
        # Fixed feedback weight
        m = np.random.uniform(size=(D, H), low=low, high=high) / np.sqrt(D / 2.)
        self.W_fixed.append(m)
        # Input layer: gradients
        self.grads.append({key: np.zeros_like(val) for key, val in self.model[0].items()})
        # Previous output layer
        self.ys_prev.append(0.0)

        # Hidden layers: weights/ biases
        m_L = []
        for _ in range(L):
            m = dict(W=np.random.uniform(size=(H, H), low=low, high=high) / np.sqrt(H / 2.), b=np.zeros((1, H)))
            m_L.append(m)
        self.model.append(m_L)
        # Fixed feedback weight
        m_L = []
        for _ in range(L):
            m = np.random.uniform(size=(H, H), low=low, high=high) / np.sqrt(H / 2.)
            m_L.append(m)
        self.W_fixed.append(m_L)
        # Hidden layer: gradients
        grad_L = []
        for _ in range(L):
            grad_L.append({key: np.zeros_like(val) for key, val in self.model[1][0].items()})
        self.grads.append(grad_L)
        # Previous output layer
        ys_prev_L = []
        for _ in range(L):
            ys_prev_L.append(0.0)
        self.ys_prev.append(ys_prev_L)
        
        # Output layer: weights/ biases
        m = dict(W=np.random.uniform(size=(H, C), low=low, high=high) / np.sqrt(H / 2.), b=np.zeros((1, C)))
        self.model.append(m)
        # Fixed feedback weight
        m = np.random.uniform(size=(H, C), low=low, high=high) / np.sqrt(H / 2.)
        self.W_fixed.append(m)
        # Outout layer: gradients
        self.grads.append({key: np.zeros_like(val) for key, val in self.model[2].items()})
        # Previous output layer
        self.ys_prev.append(0.0)
        
    def fc_forward(self, X, W, b):
        out = (X @ W) + b
        cache = (W, X)
        return out, cache

    def fc_backward(self, dout, cache, W_fixed):
        W, X = cache

        dW = X.T @ dout
        db = np.sum(dout, axis=0).reshape(1, -1) # db_1xn
        
        # dX = dout @ W.T # Backprop
        dX = dout @ W_fixed.T # fb alignment

        return dX, dW, db

    def train_forward(self, X, train):
        caches, ys = [], []
        
        # Input layer
        y, fc_cache = self.fc_forward(X=X, W=self.model[0]['W'], b=self.model[0]['b']) # X_1xD, y_1xc
        y, _ = l.selu_forward(X=y)
#         if train:
#             y, do_cache = l.dropout_forward(X=y, p_dropout=0.95) # poisson
#             caches.append((fc_cache, do_cache)) # caches[0]
        if train:
            caches.append(fc_cache)
        ys.append(y) # ys[0]
        X = y.copy() # pass to the next layer
        
        # Hidden layers
        fc_caches, ys_L, do_caches = [], [], []
        for layer in range(self.L):
            y, fc_cache = self.fc_forward(X=X, W=self.model[1][layer]['W'], b=self.model[1][layer]['b'])
            y, _ = l.selu_forward(X=y)
#             if train:
#                 y, do_cache = l.dropout_forward(X=y, p_dropout=0.95) # poisson
#                 do_caches.append(do_cache)
            fc_caches.append(fc_cache)
            ys_L.append(y) # ys[1][layer]
            X = y.copy() # pass to next layer
#         if train:
#             caches.append((fc_caches, do_caches)) # caches[1]
        if train:
            caches.append(fc_caches) # caches[1]
        ys.append(ys_L) # ys[1]            
        
        # Output layer
        y, fc_cache = self.fc_forward(X=X, W=self.model[2]['W'], b=self.model[2]['b'])
        if train:
            y, do_cache = l.dropout_forward(X=y, p_dropout=0.95) # poisson
            caches.append((fc_cache, do_cache)) # caches[2]
        ys.append(y) # ys[2]

        return ys, caches # for backpropating the error

    def loss_function(self, y, y_train):
        
        loss = cross_entropy(y, y_train) # softmax is included
        dy = dcross_entropy(y, y_train) # dsoftmax is included
        
        return loss, dy
        
    def train_backward(self, dy, caches):
        grads = self.grads # initialized by Zero in every iteration/epoch
        
        # Output layer
        fc_cache, do_cache = caches[2]
        dy = l.dropout_backward(cache=do_cache, dout=dy)
        dX, dW, db = self.fc_backward(dout=dy, cache=fc_cache, W_fixed=self.W_fixed[2])
        dy = dX.copy()
        grads[2]['W'] = dW
        grads[2]['b'] = db

        # Hidden layer
#         fc_caches, do_caches = caches[1]
        fc_caches = caches[1]
        for layer in reversed(range(self.L)):
#             dy = l.dropout_backward(cache=do_caches[layer], dout=dy)
#             dy = l.selu_backward(cache=nl_caches[layer], dout=dy)
#             dy = np.exp(dy) #/ np.exp(dy).sum(axis=1).reshape(-1, 1) # txn
            dy *= self.ys[1][layer] - self.ys_prev[1][layer] # function derivative or dfunc
            dX, dW, db = self.fc_backward(dout=dy, cache=fc_caches[layer], W_fixed=self.W_fixed[1][layer])
            dy = dX.copy()
            grads[1][layer]['W'] = dW
            grads[1][layer]['b'] = db
        
        # Input layer
#         fc_cache, do_cache = caches[0]
        fc_cache = caches[0]
#         dy = l.dropout_backward(cache=do_cache, dout=dy)
#         dy = l.selu_backward(cache=nl_cache, dout=dy)
#         dy = np.exp(dy) #/ np.exp(dy).sum(axis=1).reshape(-1, 1) # txn
        dy *= self.ys[0] - self.ys_prev[0]
        dX, dW, db = self.fc_backward(dout=dy, cache=fc_cache, W_fixed=self.W_fixed[0])
        # dy = dX.copy()
        grads[0]['W'] = dW
        grads[0]['b'] = db

        return dX, grads
    
    def test(self, X):
        ys_logit, _ = self.train_forward(X, train=False)
        y_logit = ys_logit[2] # last layer
        
        # if self.mode == 'classification':
        y_prob = l.softmax(y_logit) # for accuracy == acc
        y_pred = np.argmax(y_prob, axis=1) # for loss ==err
        
        return y_pred, y_logit
        
    def get_minibatch(self, X, y, minibatch_size, shuffle):
        minibatches = []

        if shuffle:
            X, y = skshuffle(X, y)

        for i in range(0, X.shape[0], minibatch_size):
            X_mini = X[i:i + minibatch_size]
            y_mini = y[i:i + minibatch_size]
            minibatches.append((X_mini, y_mini))

        return minibatches

    def sgd(self, train_set, val_set, alpha, mb_size, n_iter, print_after):
        X_train, y_train = train_set
        X_val, y_val = val_set

        # Epochs
        for iter in range(1, n_iter + 1):

            # Minibatches
            minibatches = self.get_minibatch(X_train, y_train, mb_size, shuffle=True)
            idx = np.random.randint(0, len(minibatches))
            X_mini, y_mini = minibatches[idx]
            
            # Train the model
            self.ys, caches = self.train_forward(X_mini, train=True)
#             print(self.ys[2].shape)
            loss, dy = self.loss_function(self.ys[2], y_mini)
            _, grads = self.train_backward(dy, caches) # self.ys_prev is used here for dfunc/ diff
            self.ys_prev = self.ys # for next iteration or epoch
            self.losses['train'].append(loss)
            
            # Update the model for input layer
            for key in grads[0].keys():
                self.model[0][key] -= alpha * grads[0][key]

            # Update the model for the hidden layers
            for layer in range(self.L):
                for key in grads[1][layer].keys():
                    self.model[1][layer][key] -= alpha * grads[1][layer][key]

            # Update the model for output layer
            for key in grads[2].keys():
                self.model[2][key] -= alpha * grads[2][key]
                
            # Validate the updated model
            y_pred, y_logit = self.test(X_val)
            valid_loss, _ = self.loss_function(y_logit, y_val) # softmax is included in entropy loss function
            self.losses['valid'].append(valid_loss)
            valid_acc = np.mean(y_pred == y_val) # confusion matrix
            self.losses['valid_acc'].append(valid_acc)
            
            # Print the model info: loss & accuracy or err & acc
            if iter % print_after == 0:
                print('Iter-{} train loss: {:.4f} valid loss: {:.4f}, valid accuracy: {:.4f}'.format(
                    iter, loss, valid_loss, valid_acc))

        # Test the final model
        y_pred, y_logit = nn.test(X_test)
        loss, _ = self.loss_function(y_logit, y_test) # softmax is included in entropy loss function
        acc = np.mean(y_pred == y_test)
        print('Last iteration - Test accuracy mean: {:.4f}, std: {:.4f}, loss: {:.4f}'.format(
            acc.mean(), acc.std(), loss))

In [ ]:
# Hyper-parameters
n_iter = 100000 # number of epochs
alpha = 1e-3 # learning_rate
mb_size = 50 # 2**10==1024 # width, timestep for sequential data or minibatch size
print_after = 100 # n_iter//10 # print loss for train, valid, and test
num_hidden_units = 32 # number of kernels/ filters in each layer
num_input_units = X_train.shape[1] # noise added at the input lavel as input noise we can use dX or for more improvement
num_output_units = y_train.max() + 1 # number of classes in this classification problem
num_layers = 2 # depth 

# Build the model/NN and learn it: running session.
nn = FFNN(C=num_output_units, D=num_input_units, H=num_hidden_units, L=num_layers)

nn.sgd(train_set=(X_train, y_train), val_set=(X_val, y_val), mb_size=mb_size, alpha=alpha, 
           n_iter=n_iter, print_after=print_after)

Iter-100 train loss: 2.3149 valid loss: 2.3169, valid accuracy: 0.1036
Iter-200 train loss: 2.3083 valid loss: 2.3165, valid accuracy: 0.1038
Iter-300 train loss: 2.3061 valid loss: 2.3162, valid accuracy: 0.1040
Iter-400 train loss: 2.3044 valid loss: 2.3159, valid accuracy: 0.1044
Iter-500 train loss: 2.3039 valid loss: 2.3155, valid accuracy: 0.1046
Iter-600 train loss: 2.2960 valid loss: 2.3151, valid accuracy: 0.1046
Iter-700 train loss: 2.3109 valid loss: 2.3147, valid accuracy: 0.1046
Iter-800 train loss: 2.3041 valid loss: 2.3143, valid accuracy: 0.1046
Iter-900 train loss: 2.3007 valid loss: 2.3140, valid accuracy: 0.1046
Iter-1000 train loss: 2.3120 valid loss: 2.3137, valid accuracy: 0.1048
Iter-1100 train loss: 2.2916 valid loss: 2.3133, valid accuracy: 0.1048
Iter-1200 train loss: 2.3091 valid loss: 2.3129, valid accuracy: 0.1050
Iter-1300 train loss: 2.2946 valid loss: 2.3126, valid accuracy: 0.1056
Iter-1400 train loss: 2.3088 valid loss: 2.3122, valid accuracy: 0.1056
I

Iter-11500 train loss: 2.3015 valid loss: 2.2786, valid accuracy: 0.1404
Iter-11600 train loss: 2.2951 valid loss: 2.2783, valid accuracy: 0.1406
Iter-11700 train loss: 2.3049 valid loss: 2.2780, valid accuracy: 0.1416
Iter-11800 train loss: 2.2989 valid loss: 2.2777, valid accuracy: 0.1426
Iter-11900 train loss: 2.3081 valid loss: 2.2774, valid accuracy: 0.1428
Iter-12000 train loss: 2.3098 valid loss: 2.2770, valid accuracy: 0.1438
Iter-12100 train loss: 2.3014 valid loss: 2.2767, valid accuracy: 0.1444
Iter-12200 train loss: 2.3171 valid loss: 2.2764, valid accuracy: 0.1448
Iter-12300 train loss: 2.3075 valid loss: 2.2760, valid accuracy: 0.1454
Iter-12400 train loss: 2.3021 valid loss: 2.2756, valid accuracy: 0.1460
Iter-12500 train loss: 2.3024 valid loss: 2.2753, valid accuracy: 0.1460
Iter-12600 train loss: 2.3110 valid loss: 2.2750, valid accuracy: 0.1462
Iter-12700 train loss: 2.2906 valid loss: 2.2747, valid accuracy: 0.1462
Iter-12800 train loss: 2.2969 valid loss: 2.2743, v

Iter-22800 train loss: 2.3076 valid loss: 2.2431, valid accuracy: 0.1814
Iter-22900 train loss: 2.3056 valid loss: 2.2428, valid accuracy: 0.1818
Iter-23000 train loss: 2.3059 valid loss: 2.2425, valid accuracy: 0.1828
Iter-23100 train loss: 2.3009 valid loss: 2.2422, valid accuracy: 0.1828
Iter-23200 train loss: 2.2896 valid loss: 2.2419, valid accuracy: 0.1830
Iter-23300 train loss: 2.3064 valid loss: 2.2415, valid accuracy: 0.1828
Iter-23400 train loss: 2.3049 valid loss: 2.2413, valid accuracy: 0.1832
Iter-23500 train loss: 2.2966 valid loss: 2.2409, valid accuracy: 0.1830
Iter-23600 train loss: 2.3010 valid loss: 2.2406, valid accuracy: 0.1832
Iter-23700 train loss: 2.2932 valid loss: 2.2403, valid accuracy: 0.1836
Iter-23800 train loss: 2.3096 valid loss: 2.2400, valid accuracy: 0.1840
Iter-23900 train loss: 2.3011 valid loss: 2.2397, valid accuracy: 0.1844
Iter-24000 train loss: 2.2952 valid loss: 2.2394, valid accuracy: 0.1846
Iter-24100 train loss: 2.3040 valid loss: 2.2391, v

In [ ]:
# # Display the learning curve and losses for training, validation, and testing
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

plt.plot(nn.losses['train'], label='Train loss')
plt.plot(nn.losses['valid'], label='Valid loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(nn.losses['valid_acc'], label='Valid accuracy')
plt.legend()
plt.show()